In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm.notebook import tqdm
from lightKNN import LightKNN
from collections import Counter

In [2]:
%%time
song = pd.read_json("res/song_meta.json")[["id", "song_name", "artist_id_basket", "artist_name_basket",\
                                            "album_id", "album_name", "song_gn_gnr_basket",\
                                            "song_gn_dtl_gnr_basket", "issue_date"]]
genre = pd.read_json("res/genre_gn_all.json", typ="series")
train = pd.read_json("res/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val = pd.read_json("res/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
test = pd.read_json("res/test.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]

train80 = pd.read_json("arena_data/orig/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val20q = pd.read_json("arena_data/questions/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val20a = pd.read_json("arena_data/answers/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
result_example = pd.read_json("arena_data/results/results_1.json")

display(song, genre, train, val, test)
display(train80, val20q, val20a, result_example)

,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,issue_date
0,0,Feelings,[2727],[Various Artists],2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[GN0900],[GN0901],20140512
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",20080421
2,2,Solsbury Hill (Remastered 2002),[3361],[Peter Gabriel],4698747,Hit,[GN0900],[GN0901],20180518
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[838543],[Matoma],2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],"[GN1102, GN1101]",20151016
4,4,그남자 그여자,[560160],[Jude Law],2008470,그남자 그여자,[GN1800],"[GN1802, GN1801]",20110824
...,...,...,...,...,...,...,...,...,...
707984,707984,Coffin For Head Of State,[166499],[Fela Kuti],65254,The Best Best Of The Black President,[GN2000],[GN2001],19991219
707985,707985,Change Of Heart,[11837],[Cyndi Lauper],44141,True Colors,[GN0900],[GN0901],19860000
707986,707986,스치듯 안녕,[437],[윤종신],2662866,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,[GN0100],"[GN0105, GN0101]",20160120
707987,707987,숲의 빛,[729868],[Nature Piano],2221722,명상의 시간을 위한 뉴에이지 음악,[GN1800],"[GN1807, GN1801]",20131217


GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Length: 254, dtype: object

,id,plylst_title,songs,tags,like_cnt,updt_date
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",17,2017-08-28 07:09:34.000
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33,2019-12-05 15:15:18.000
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3,2020-04-17 04:31:11.000
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",[일렉],13,2015-12-24 17:23:19.000
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4,2019-08-16 20:59:22.000
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4,2019-10-25 23:40:42.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],1675,2019-05-27 14:14:33.000
1,131447,앨리스테이블,[],[],1,2014-07-16 15:24:24.000
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],62,2008-06-21 23:26:22.000
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],20,2017-10-30 18:15:43.000
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한],17,2015-12-17 14:06:05.000
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",10,2020-04-16 21:35:44.000
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[],0,2019-03-27 15:27:40.000
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[],31,2015-11-18 11:49:09.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],6,2012-09-29 01:57:26.000
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[나만의Best3, 인디아티스트들의추천음악]",0,2019-12-17 14:06:45.000
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브],21,2015-05-23 10:44:48.000
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기],0,2019-03-14 09:47:34.000
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[],38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억],16,2016-04-18 11:02:09.000
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡의, 우울, 분위기, 드라이브, 산책]",1,2020-04-08 07:15:59.000
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",[기분전환],6,2016-06-29 00:57:21.000
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름],4,2015-06-06 09:52:01.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...","[힐링, 휴식, 밤, 새벽]",12,2016-06-23 10:06:27.000
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",[팝],0,2013-08-15 13:17:11.000
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",[뉴에이지],23,2015-09-03 16:51:50.000
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...","[하드락, 록스피릿, 댄스]",1,2017-01-09 15:41:25.000
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...","[힐링, 휴식, 기분전환]",715,2016-02-22 12:32:50.000
...,...,...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...","[90년생, 회상, 추억, 좋은노래, 80년생, 옛날노래]",155,2020-01-15 15:15:45.000
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",[팝],8,2010-03-23 00:03:00.000
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...","[여행, 발라드, 기분전환, 사랑]",3,2019-05-15 13:26:07.000
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...","[소울, 알앤비]",51,2013-12-24 14:40:01.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,119064,재즈! 신나는재즈!,[],[],18,2017-05-03 17:18:07.000
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[274165, 552540, 104677, 188486, 271300, 25864...",[드라이브],29,2019-10-17 09:32:41.000
2,68628,ⓗ 감성가요,"[275414, 477905, 256688, 276685, 109237, 54408...",[랩],0,2013-12-03 11:51:26.000
3,100993,위로의 형태,[],[],2,2018-03-12 11:23:21.000
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[632115, 307581, 357367, 112832, 648635, 60663...",[],612,2019-03-06 18:35:20.000
...,...,...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[222048, 562040, 295682, 474983, 405990, 61620...",[잔잔한],6,2016-09-15 12:05:43.000
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[158590, 375777, 598734, 63734, 112399, 76888,...",[힐링],7,2015-01-01 11:13:45.000
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,[],[],38,2011-01-06 00:04:33.000
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[353960, 221785, 234462, 22976, 31784, 403481,...",[],0,2015-05-05 13:29:48.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,119064,재즈! 신나는재즈!,"[231820, 13676, 593705, 23746, 429366, 376182,...",[재즈],18,2017-05-03 17:18:07.000
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[448698, 156706, 185733, 117595, 497066, 28628...",[가을],29,2019-10-17 09:32:41.000
2,68628,ⓗ 감성가요,"[407073, 529965, 700041, 157285, 578509, 65827...",[힙합],0,2013-12-03 11:51:26.000
3,100993,위로의 형태,"[445568, 114172, 234233, 348174, 521628, 43010...",[슬픔],2,2018-03-12 11:23:21.000
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[468023, 637294, 130632, 340538, 290477, 39356...","[위로, 혼자있고싶을때, 따뜻한]",612,2019-03-06 18:35:20.000
...,...,...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[335474, 105039, 401182, 398584, 311731, 51024...",[까페],6,2016-09-15 12:05:43.000
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[370002, 551409, 95323, 346969, 579728, 60117,...","[휴식, 잔잔한]",7,2015-01-01 11:13:45.000
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,"[572238, 174778, 198528, 280899, 310653, 41008...",[팝],38,2011-01-06 00:04:33.000
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[315273, 245317, 359517, 593278, 594982, 22588...",[발라드],0,2015-05-05 13:29:48.000


,id,songs,tags
0,119064,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
1,64891,"[144663, 357367, 366786, 654757, 133143, 67511...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 힐링, 사랑, 새벽, 밤, 카페]"
2,68628,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
3,100993,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
4,10417,"[144663, 116573, 366786, 654757, 133143, 67511...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
...,...,...,...
23010,115244,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 드라이브, 힐링, 사랑, 새벽, 밤, 카페]"
23011,124493,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 사랑, 새벽, 밤, 카페]"
23012,65279,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
23013,53828,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"


CPU times: user 13.8 s, sys: 1.16 s, total: 14.9 s
Wall time: 15 s


In [3]:
with open("bin/Xs.p", 'rb') as f:
    Xs = pickle.load(f)
x = Xs[0]
X = Xs[1]
display(x, X)

,id,plylst_title,songs,tags
0,147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...","[새벽, 밤, 휴식, 힐링]"
1,50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",[팝]
2,116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...","[뉴에이, 숙면, 마음, 피아노, 뉴에이지]"
3,55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...","[록스피릿, 하드락, 곡, 댄스, 울]"
4,125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...","[음악, 필수, 힐링, 힙합, 스피커, 기분전환, 전환, 휴식, 기분]"
...,...,...,...,...
92051,149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...","[좋은노래, 회상, 모음, 년생, 년, 노래, 옛날노래, 추억]"
92052,35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",[팝]
92053,59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...","[발라드, 베스트, 사랑, 기분, 기분전환, 전환, 여행, 추억]"
92054,9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...","[알앤비, 선정, 소울]"


,id,plylst_title,songs,tags
0,119064,재즈! 신나는재즈!,[],"[신나는, 재즈, 즈]"
1,64891,"가을의 문턱에서! anywhere, anybody :) 듣기좋은 음악","[274165, 552540, 104677, 188486, 271300, 25864...","[가을, 문턱, 드라이브, 음악]"
2,68628,ⓗ 감성가요,"[275414, 477905, 256688, 276685, 109237, 54408...","[랩, 감성가]"
3,100993,위로의 형태,[],"[위로, 형태]"
4,10417,"조금 지친 밤, 눈 감고 듣고 싶은 위로의 음악","[632115, 307581, 357367, 112832, 648635, 60663...","[눈, 밤, 위로, 음악]"
...,...,...,...,...
23010,115244,밤하늘 보며 듣고 싶은 잔잔한 음악,"[222048, 562040, 295682, 474983, 405990, 61620...","[밤하늘, 잔잔한, 음악]"
23011,124493,MBC FM4U 라디오 14.12.01 ~ 31 : 그 달의 음악,"[158590, 375777, 598734, 63734, 112399, 76888,...","[라디오, 음악, 달, 힐링]"
23012,65279,한겨울의 따뜻한 핫초코 같은 음악,[],"[겨울, 음악, 핫초코]"
23013,53828,우리 함께 했던 시간들이 담긴 음악,"[353960, 221785, 234462, 22976, 31784, 403481,...","[음악, 시간]"


In [3]:
%%time
# Latest Submission
k = 100
rho = 0.4
knn = LightKNN(k, rho=rho, sim="cosine")
knn.fit(train)
pred = knn.predict(val, end=None)
pred.to_json('submission/lightknn_100_cos.json', orient='records')
pred

100%|██████████| 23015/23015 [1:31:14<00:00,  4.20it/s]


CPU times: user 1h 30min 44s, sys: 34 s, total: 1h 31min 18s
Wall time: 1h 31min 15s


,id,songs,tags
0,118598,"[207912, 623047, 422438, 703323, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 디즈니OST, 기분전환, 추억, 영화OST,..."
1,131447,"[16382, 349306, 660725, 595188, 283888, 496879...","[기분전환, 발라드, 드라이브, 힙합, 힐링, 카페, 휴식, 랩, 추억, 감성]"
2,51464,"[533739, 456474, 680911, 596279, 214521, 65575...","[발라드, 추억, 회상, 이별, 슬픔, 설렘, 밤, 새벽, 사랑, 알앤비]"
3,45144,"[144663, 367963, 174749, 377243, 7070, 543820,...","[발라드, 감성, 이별, 새벽, 잔잔한, 기분전환, 사랑, 카페, 추억, 밤]"
4,79929,"[211220, 412769, 445299, 106129, 70314, 242124...","[CCM, 찬양, 은혜, 예배, 사랑, 국내ccm, 교회, 파워DJ_CCM, 은혜로..."
...,...,...,...
23010,101722,"[339802, 116573, 675115, 473514, 449244, 87504...","[이별, 발라드, 슬픔, 새벽, 밤, 휴식, 추억, 감성, 사랑, 기분전환]"
23011,122127,"[352228, 138932, 4173, 473514, 630552, 48209, ...","[사랑, 추억, 회상, 부모님, 엄마, 휴식, 힐링, 감성, 발라드, 잔잔한]"
23012,77438,"[679436, 274504, 140867, 459256, 696317, 57101...","[팝, Pop, 팝송, 기분전환, 드라이브, 카페, 휴식, 매장음악, 신나는, 여행]"
23013,36231,"[64680, 354737, 651788, 375318, 532729, 81796,...","[클래식, 잔잔한, 힐링, 휴식, 팝송, 감성, 기분전환, 팝, 피아노, Pop]"


In [4]:
%%time
k = 100
rho = 0.4
knn = LightKNN(k, rho=rho, sim="idf")
knn.fit(train)
pred = knn.predict(val, end=100)
pred.to_json('submission/lightknn_100_idf_0.4.json', orient='records')
pred

100%|██████████| 100/100 [02:39<00:00,  1.60s/it]

CPU times: user 2min 38s, sys: 12.6 s, total: 2min 51s
Wall time: 2min 42s


,id,songs,tags
0,118598,"[207912, 623047, 422438, 703323, 638488, 32221...","[OST, 디즈니, 애니메이션, 영화, 추억, 기분전환, 동심, 디즈니OST, 픽사..."
1,131447,"[16382, 349306, 660725, 595188, 283888, 496879...","[기분전환, 발라드, 드라이브, 힙합, 힐링, 카페, 휴식, 랩, 추억, 감성]"
2,51464,"[533739, 214521, 610218, 457500, 456474, 40277...","[발라드, 추억, 회상, 이별, 슬픔, 설렘, 밤, 새벽, 사랑, 감성]"
3,45144,"[144663, 367963, 174749, 224921, 116114, 19533...","[발라드, 감성, 사랑, 잔잔한, 이별, 새벽, 카페, 기분전환, 휴식, 밤]"
4,79929,"[211220, 412769, 445299, 106129, 70314, 526037...","[CCM, 찬양, 은혜, 사랑, 예배, 국내ccm, 교회, 찬송가, 위로, 파워DJ..."
...,...,...,...
95,19457,"[375431, 347585, 553166, 277034, 443914, 13319...","[댄스, 아이돌, 기분전환, 걸그룹, 신나는, 드라이브, 운동, 여자아이돌, 스트레..."
96,129612,"[16382, 349306, 660725, 595188, 283888, 496879...","[기분전환, 발라드, 드라이브, 힙합, 카페, 휴식, 랩, 추억, 감성, 잔잔한]"
97,829,"[698012, 524652, 527560, 126799, 447054, 54953...","[재즈, 힐링, 휴식, 가을, 새벽, 기분전환, 밤, 팝, 감성, 잔잔한]"
98,145963,"[671426, 660381, 260730, 23276, 76676, 583569,...","[댄스, 추억, 기분전환, 아이돌, 회상, 신나는, 운동, 드라이브, 발라드, 20..."
